# 子词嵌入
:label:`sec_fasttext`

在英语中，“helps”、“helped”和“helping”都是同一个词“help”的屈折形式。“dog”和“dogs”之间的关系与“cat”和“cats”之间的关系相同，而“boy”和“boyfriend”之间的关系与“girl”和“girlfriend”之间的关系相同。在法语和西班牙语等其他语言中，许多动词有超过40种屈折形式，而在芬兰语中，一个名词可能有高达15种格。在语言学中，形态学研究词的构成和词的关系。然而，在word2vec和GloVe中都没有探索词的内部结构。

## fastText模型

回想一下单词在word2vec中的表示方式。在跳字模型和连续词袋模型中，同一单词的不同屈折形式直接由不同的向量表示，没有共享参数。为了利用形态信息，*fastText*模型提出了一种*子词嵌入*方法，其中子词是一个字符$n$-gram :cite:`Bojanowski.Grave.Joulin.ea.2017`。与其学习词级向量表示，不如将fastText视为子词级别的跳字模型，其中每个*中心词*由其子词向量之和表示。

让我们以单词“where”为例说明如何在fastText中获取每个中心词的子词。首先，在单词的开头和结尾添加特殊字符“&lt;”和“&gt;”，以区分前缀和后缀与其他子词。然后，从单词中提取字符$n$-gram。例如，当$n=3$时，我们得到所有长度为3的子词：“&lt;wh”，“whe”，“her”，“ere”，“re&gt;”，以及特殊的子词“&lt;where&gt;”。

在fastText中，对于任何单词$w$，记$\mathcal{G}_w$为其所有长度在3到6之间及其特殊子词的并集。词汇表是所有单词的子词的并集。设$\mathbf{z}_g$为词典中子词$g$的向量，则作为跳字模型中中心词的单词$w$的向量$\mathbf{v}_w$是其子词向量的总和：

$$\mathbf{v}_w = \sum_{g\in\mathcal{G}_w} \mathbf{z}_g.$$

fastText的其余部分与跳字模型相同。与跳字模型相比，fastText的词汇量更大，导致更多的模型参数。此外，为了计算一个单词的表示，必须将其所有子词向量相加，从而导致更高的计算复杂度。然而，由于具有相似结构的单词之间子词参数的共享，罕见词甚至词汇外的词在fastText中可能获得更好的向量表示。

## 字节对编码
:label:`subsec_Byte_Pair_Encoding`

在fastText中，所有提取的子词都必须是指定的长度，如3到6，因此词汇量大小不能预先定义。为了在一个固定大小的词汇表中允许变长的子词，我们可以应用一种称为*字节对编码*（BPE）的压缩算法来提取子词 :cite:`Sennrich.Haddow.Birch.2015`。

字节对编码通过对训练数据集进行统计分析来发现单词内的常见符号，如任意长度的连续字符。从长度为1的符号开始，字节对编码迭代地合并最频繁出现的一对连续符号以产生新的更长符号。注意，出于效率考虑，不考虑跨越词边界的配对。最终，我们可以使用这些符号作为子词来分割单词。字节对编码及其变体已被用于流行自然语言处理预训练模型的输入表示，如GPT-2 :cite:`Radford.Wu.Child.ea.2019`和RoBERTa :cite:`Liu.Ott.Goyal.ea.2019`。下面我们将说明字节对编码的工作原理。

首先，我们将符号词汇初始化为所有英文小写字母、一个特殊的词尾符号`'_'`和一个特殊的未知符号`'[UNK]'`。

In [1]:
import collections

symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           '_', '[UNK]']

由于我们不考虑跨越单词边界的符号对，所以我们只需要一个字典`raw_token_freqs`，它将单词映射到它们在数据集中的频率（出现次数）。请注意，每个单词后面都附加了特殊符号`'_'`，这样我们可以轻松地从输出符号序列（例如，“a_ tall er_ man”）中恢复单词序列（例如，“a taller man”）。由于我们从仅包含单个字符和特殊符号的词汇表开始合并过程，所以在字典`token_freqs`的每个单词内的每对连续字符之间插入空格。换句话说，空格是单词内符号之间的分隔符。

In [2]:
raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
for token, freq in raw_token_freqs.items():
    token_freqs[' '.join(list(token))] = raw_token_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}

我们定义了以下`get_max_freq_pair`函数，该函数返回输入字典`token_freqs`的键中的单词内出现频率最高的连续符号对。

In [3]:
def get_max_freq_pair(token_freqs):
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # Key of `pairs` is a tuple of two consecutive symbols
            pairs[symbols[i], symbols[i + 1]] += freq
    return max(pairs, key=pairs.get)  # Key of `pairs` with the max value

作为一种基于连续符号频率的贪婪方法，字节对编码将使用以下`merge_symbols`函数来合并最频繁出现的连续符号对以生成新的符号。

In [4]:
def merge_symbols(max_freq_pair, token_freqs, symbols):
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

现在我们迭代地对字典 `token_freqs` 的键执行字节对编码算法。在第一次迭代中，最频繁出现的连续符号对是 `'t'` 和 `'a'`，因此字节对编码将它们合并生成一个新的符号 `'ta'`。在第二次迭代中，字节对编码继续合并 `'ta'` 和 `'l'` 以产生另一个新符号 `'tal'`。

In [5]:
num_merges = 10
for i in range(num_merges):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print(f'merge #{i + 1}:', max_freq_pair)

merge #1: ('t', 'a')
merge #2: ('ta', 'l')
merge #3: ('tal', 'l')
merge #4: ('f', 'a')
merge #5: ('fa', 's')
merge #6: ('fas', 't')
merge #7: ('e', 'r')
merge #8: ('er', '_')
merge #9: ('tall', '_')
merge #10: ('fast', '_')


经过10次字节对编码迭代后，我们可以看到列表`symbols`现在包含了另外10个从其他符号迭代合并而来的新符号。

In [6]:
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']


对于字典 `raw_token_freqs` 中指定的同一数据集，数据集中的每个单词现在通过子词“fast_”、“fast”、“er_”、“tall_”和“tall”进行了分割，这是字节对编码算法的结果。例如，单词“faster_”和“taller_”分别被分割为“fast er_”和“tall er_”。

In [7]:
print(list(token_freqs.keys()))

['fast_', 'fast er_', 'tall_', 'tall er_']


请注意，字节对编码的结果取决于所使用的数据集。
我们也可以使用从一个数据集中学到的子词
来分割另一个数据集中的词。
作为一种贪心方法，下面的 `segment_BPE` 函数试图将词分解成从输入参数 `symbols` 中最长可能的子词。

In [8]:
def segment_BPE(tokens, symbols):
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        # Segment token with the longest possible subwords from symbols
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        if start < len(token):
            cur_output.append('[UNK]')
        outputs.append(' '.join(cur_output))
    return outputs

在下面的内容中，我们使用从上述数据集中学习到的`symbols`列表中的子词来分割代表另一个数据集的`tokens`。

In [9]:
tokens = ['tallest_', 'fatter_']
print(segment_BPE(tokens, symbols))

['tall e s t _', 'fa t t er_']


## 摘要

* fastText模型提出了一种子词嵌入方法。基于word2vec中的skip-gram模型，它将中心词表示为其子词向量的总和。
* 字节对编码通过对训练数据集进行统计分析来发现单词内的常见符号。作为一种贪婪方法，字节对编码迭代地合并最频繁出现的连续符号对。
* 子词嵌入可能提高稀有词和词典外词汇表示的质量。

## 练习

1. 例如，英语中大约有$3\times 10^8$个可能的$6$-grams。当子词过多时会出现什么问题？如何解决这个问题？提示：参考fastText论文 :cite:`Bojanowski.Grave.Joulin.ea.2017`第3.2节末尾。
1. 如何基于连续词袋模型设计一个子词嵌入模型？
1. 当初始符号词汇大小为$n$时，为了得到大小为$m$的词汇表，需要多少次合并操作？
1. 如何扩展字节对编码的思想以提取短语？

[讨论](https://discuss.d2l.ai/t/4587)